# Testing single node datasets for normalized single node behaviors

Purpose: until now dataset represented by combination of tree types - now test for single node type

## Importing

In [1]:
import sys
import os

# Construct the path to the parent directory of the parent directory
parent_of_parent_dir = os.path.abspath(os.path.join(os.path.pardir, os.path.pardir))

# Add the parent directory of the parent directory to the system path
sys.path.insert(0, parent_of_parent_dir)

In [2]:
import optuna
import numpy as np
from sklearn.metrics import r2_score
import torch
from torch.utils.data import random_split, DataLoader

from etnn import TreeNode
from etnn.tools.training_tools import ConfigStore
from etnn.data.ferris_wheel import load_pure_ferris_wheel_dataset_single_node
from etnn.tools.training import train_epoch, eval_epoch
from etnn.nn.layer_framework import LayerManagementFramework
from etnn.tools.training_tools import seeding_all
from etnn.routines.run_config import choice_trainloader, choice_loss, choice_optim

## Control constants

In [3]:
dataset_path = "../../datasets/"
test_perc = 0.3
val_perc = 0.21
stability_count = 5
label = "tree-advanced" # although irrelevant

## Defining parameter search

In [4]:
def objective(trial):
    # init default config
    config = ConfigStore(
       in_dim=15,
       hidden_dim=trial.suggest_int("hidden_dim", 16, 512, step=16),
       out_dim=1,
       k=trial.suggest_int("k", 1, 5),
       dataset=-1 if normalized else 0,
       ds_size=dataset_size,
       num_gondolas=-1,
       num_part_pg=-1,
       loss_name='mse',
       optimizer_name='adam',
       num_max_epochs=30, # real: 100
       learning_rate=trial.suggest_float("learning_rate", 1e-5, 1e-2, log=True),
       batch_size=1024,
       early_stop_tol=5,
       use_equal_batcher=trial.suggest_categorical("batcher", [True, False]),
       seed=420,
       label_type=label,
       final_label_factor=5/1000
    )
    # config = ConfigStore(
    #     in_dim=15,
    #     hidden_dim=16,
    #     out_dim=1,
    #     k=2,
    #     dataset=-1 if normalized else 0,
    #     ds_size=10_000,
    #     num_gondolas=-1,
    #     num_part_pg=-1,
    #     loss_name='mse',
    #     optimizer_name='adam',
    #     num_max_epochs=30, # real: 100
    #     learning_rate=0.002,
    #     batch_size=1024,
    #     early_stop_tol=5,
    #     use_equal_batcher=False,
    #     seed=420,
    #     label_type=label,
    #     final_label_factor=5/1000
    # )

    # loading dataset
    dataset, df_index = load_pure_ferris_wheel_dataset_single_node(
        node_type=node_type,
        num_elem=size_elem,
        num_to_generate=config.ds_size,
        dataset_path=dataset_path,
        final_label_factor=config.final_label_factor,
        normalize=True if config.dataset == -1 else 0
    )

    if post_normalized:
        dataset.post_normalize()

    # splitting off test dataset
    generator = torch.Generator().manual_seed(config.seed)
    train_ds, val_ds, _ = random_split(
        dataset,
        [1 - test_perc - val_perc, val_perc, test_perc],
        generator=generator
    )

    # loaders
    train_loader = choice_trainloader(config, df_index, train_ds)
    val_loader = DataLoader(val_ds, batch_size=4 * config.batch_size, shuffle=False)

    # define device
    device = "cuda" if torch.cuda.is_available() else "cpu"

    # set seed for reproducability
    seeding_all(config.seed)

    # define model
    model = LayerManagementFramework(
        in_dim=config.in_dim,
        tree=TreeNode(node_type, [TreeNode("E", size_elem)]),
        hidden_dim=config.hidden_dim,
        out_dim=config.out_dim,
        k=config.k
    ).to(device)

    # learning tools
    criterion = choice_loss(config)
    optimizer = choice_optim(config, model)

    # init score list
    score_list = []

    # train for specified number of epochs
    for epoch in range(config.num_max_epochs):
        _, _, _ = train_epoch(
            model,
            train_loader,
            optimizer,
            device,
            criterion
        )

        _, val_true_y, val_pred_y = eval_epoch(
            model,
            val_loader,
            device,
            criterion
        )

        # calc r2 score and append
        score = r2_score(y_true=val_true_y, y_pred=val_pred_y)
        score_list += [score]
        trial.report(score, epoch)

    # calculate objective
    # display(score_list)
    # idea: last x r2 scores (why not last one? for stability purposes)
    obj = np.array(score_list)[-stability_count:]
    return np.mean(obj)

## Parameter search

### S

In [5]:
node_type = "S"

#### Data set size 10

In [6]:
size_elem = 10
n_trials = 50
dataset_size = 10
normalized = False
post_normalized = False
study = optuna.create_study(study_name=f"Study node type {node_type}, normalized: {normalized}, post-normalized:{post_normalized}, dataset size: {dataset_size}", directions=['maximize'])
study.optimize(objective, n_trials=n_trials, show_progress_bar=True)

[I 2023-12-27 16:21:03,353] A new study created in memory with name: Study node type S, normalized: False, post-normalized:False, dataset size: 10


  0%|          | 0/50 [00:00<?, ?it/s]


100%|██████████| 10/10 [00:00<?, ?it/s][A

100%|██████████| 10/10 [00:00<00:00, 80.00it/s][A


[I 2023-12-27 16:21:07,552] Trial 0 finished with value: -114.66607971382018 and parameters: {'hidden_dim': 416, 'k': 1, 'learning_rate': 2.5778123658416155e-05, 'batcher': True}. Best is trial 0 with value: -114.66607971382018.
[I 2023-12-27 16:21:07,786] Trial 1 finished with value: -1699.3802128386742 and parameters: {'hidden_dim': 368, 'k': 2, 'learning_rate': 2.2667123720346917e-05, 'batcher': True}. Best is trial 0 with value: -114.66607971382018.
[I 2023-12-27 16:21:07,963] Trial 2 finished with value: -49.438062331052365 and parameters: {'hidden_dim': 144, 'k': 1, 'learning_rate': 0.00011122872711931957, 'batcher': True}. Best is trial 2 with value: -49.438062331052365.
[I 2023-12-27 16:21:08,168] Trial 3 finished with value: -15886.441591873994 and parameters: {'hidden_dim': 416, 'k': 2, 'learning_rate': 0.003097790284821405, 'batcher': True}. Best is trial 2 with value: -49.438062331052365.
[I 2023-12-27 16:21:08,384] Trial 4 finished with value: -2865.505469235683 and parame

In [7]:
study.trials_dataframe().to_csv(f"csvs/single-node_t-{node_type}_n-{size_elem}_normalized-{normalized}_post-normalized-{post_normalized}_dataset-size-{dataset_size}.csv")

In [8]:
size_elem = 10
n_trials = 200
dataset_size = 10
normalized = True
post_normalized = False
study = optuna.create_study(study_name=f"Study node type {node_type}, normalized: {normalized}, post-normalized:{post_normalized}, dataset size: {dataset_size}", directions=['maximize'])
study.optimize(objective, n_trials=n_trials, show_progress_bar=True)

[I 2023-12-27 16:21:18,178] A new study created in memory with name: Study node type S, normalized: True, post-normalized:False, dataset size: 10


  0%|          | 0/200 [00:00<?, ?it/s]


100%|██████████| 10/10 [00:00<00:00, 10005.50it/s]

100%|██████████| 10/10 [00:00<00:00, 117.64it/s]


[I 2023-12-27 16:21:21,657] Trial 0 finished with value: -112896.9757383123 and parameters: {'hidden_dim': 144, 'k': 2, 'learning_rate': 6.7745430706583e-05, 'batcher': True}. Best is trial 0 with value: -112896.9757383123.
[I 2023-12-27 16:21:22,103] Trial 1 finished with value: -82459.67452473532 and parameters: {'hidden_dim': 384, 'k': 2, 'learning_rate': 3.666591261924824e-05, 'batcher': False}. Best is trial 1 with value: -82459.67452473532.
[I 2023-12-27 16:21:22,333] Trial 2 finished with value: -126532.34164453109 and parameters: {'hidden_dim': 48, 'k': 5, 'learning_rate': 0.00010720947232565227, 'batcher': True}. Best is trial 1 with value: -82459.67452473532.
[I 2023-12-27 16:21:22,580] Trial 3 finished with value: -111715.6777065349 and parameters: {'hidden_dim': 128, 'k': 3, 'learning_rate': 8.715713497478029e-05, 'batcher': False}. Best is trial 1 with value: -82459.67452473532.
[I 2023-12-27 16:21:22,870] Trial 4 finished with value: -132368.0410603247 and parameters: {'h

In [9]:
study.trials_dataframe().to_csv(f"csvs/single-node_t-{node_type}_n-{size_elem}_normalized-{normalized}_post-normalized-{post_normalized}_dataset-size-{dataset_size}.csv")

In [10]:
size_elem = 10
n_trials = 200
dataset_size = 10
normalized = False
post_normalized = True
study = optuna.create_study(study_name=f"Study node type {node_type}, normalized: {normalized}, post-normalized:{post_normalized}, dataset size: {dataset_size}", directions=['maximize'])
study.optimize(objective, n_trials=n_trials, show_progress_bar=True)

[I 2023-12-27 16:22:08,065] A new study created in memory with name: Study node type S, normalized: False, post-normalized:True, dataset size: 10


  0%|          | 0/200 [00:00<?, ?it/s]

[I 2023-12-27 16:22:08,321] Trial 0 finished with value: -114.99304959726385 and parameters: {'hidden_dim': 448, 'k': 4, 'learning_rate': 4.6300707962508605e-05, 'batcher': True}. Best is trial 0 with value: -114.99304959726385.
[I 2023-12-27 16:22:08,562] Trial 1 finished with value: -1.9562479797977805 and parameters: {'hidden_dim': 480, 'k': 3, 'learning_rate': 0.0010809385240710104, 'batcher': False}. Best is trial 1 with value: -1.9562479797977805.
[I 2023-12-27 16:22:08,805] Trial 2 finished with value: -0.09451762239439336 and parameters: {'hidden_dim': 496, 'k': 3, 'learning_rate': 0.0014518370115103767, 'batcher': True}. Best is trial 2 with value: -0.09451762239439336.
[I 2023-12-27 16:22:09,027] Trial 3 finished with value: -15.379450738250984 and parameters: {'hidden_dim': 240, 'k': 4, 'learning_rate': 0.0004231026009208048, 'batcher': False}. Best is trial 2 with value: -0.09451762239439336.
[I 2023-12-27 16:22:09,231] Trial 4 finished with value: -1.875586321349866 and pa

In [11]:
study.trials_dataframe().to_csv(f"csvs/single-node_t-{node_type}_n-{size_elem}_normalized-{normalized}_post-normalized-{post_normalized}_dataset-size-{dataset_size}.csv")

#### Data set size 100

In [12]:
size_elem = 10
n_trials = 50
dataset_size = 100
normalized = False
post_normalized = False
study = optuna.create_study(study_name=f"Study node type {node_type}, normalized: {normalized}, post-normalized:{post_normalized}, dataset size: {dataset_size}", directions=['maximize'])
study.optimize(objective, n_trials=n_trials, show_progress_bar=True)

[I 2023-12-27 16:22:56,632] A new study created in memory with name: Study node type S, normalized: False, post-normalized:False, dataset size: 100


  0%|          | 0/50 [00:00<?, ?it/s]


100%|██████████| 100/100 [00:00<00:00, 50039.42it/s]

100%|██████████| 100/100 [00:03<00:00, 26.85it/s]


[I 2023-12-27 16:23:02,213] Trial 0 finished with value: -129.34703898755183 and parameters: {'hidden_dim': 416, 'k': 4, 'learning_rate': 2.5828432616403225e-05, 'batcher': False}. Best is trial 0 with value: -129.34703898755183.
[I 2023-12-27 16:23:03,166] Trial 1 finished with value: -560.5398150613557 and parameters: {'hidden_dim': 208, 'k': 5, 'learning_rate': 0.0007956707471253289, 'batcher': False}. Best is trial 0 with value: -129.34703898755183.
[I 2023-12-27 16:23:04,451] Trial 2 finished with value: -394.23955329278914 and parameters: {'hidden_dim': 464, 'k': 4, 'learning_rate': 4.511111379639611e-05, 'batcher': False}. Best is trial 0 with value: -129.34703898755183.
[I 2023-12-27 16:23:05,490] Trial 3 finished with value: -1173.9624018624681 and parameters: {'hidden_dim': 272, 'k': 4, 'learning_rate': 0.001965975236143295, 'batcher': True}. Best is trial 0 with value: -129.34703898755183.
[I 2023-12-27 16:23:06,519] Trial 4 finished with value: -1800.044847923065 and parame

In [13]:
study.trials_dataframe().to_csv(f"csvs/single-node_t-{node_type}_n-{size_elem}_normalized-{normalized}_post-normalized-{post_normalized}_dataset-size-{dataset_size}.csv")

In [14]:
size_elem = 10
n_trials = 200
dataset_size = 100
normalized = True
post_normalized = False
study = optuna.create_study(study_name=f"Study node type {node_type}, normalized: {normalized}, post-normalized:{post_normalized}, dataset size: {dataset_size}", directions=['maximize'])
study.optimize(objective, n_trials=n_trials, show_progress_bar=True)

[I 2023-12-27 16:23:50,773] A new study created in memory with name: Study node type S, normalized: True, post-normalized:False, dataset size: 100


  0%|          | 0/200 [00:00<?, ?it/s]


100%|██████████| 100/100 [00:00<00:00, 100007.25it/s]

100%|██████████| 100/100 [00:02<00:00, 33.98it/s][A


[I 2023-12-27 16:23:55,469] Trial 0 finished with value: -75.27167961882382 and parameters: {'hidden_dim': 464, 'k': 3, 'learning_rate': 3.878412386687306e-05, 'batcher': True}. Best is trial 0 with value: -75.27167961882382.
[I 2023-12-27 16:23:56,416] Trial 1 finished with value: -5.1347054137315284 and parameters: {'hidden_dim': 256, 'k': 5, 'learning_rate': 0.000341403689070165, 'batcher': False}. Best is trial 1 with value: -5.1347054137315284.
[I 2023-12-27 16:23:57,190] Trial 2 finished with value: -158.0021563481075 and parameters: {'hidden_dim': 128, 'k': 4, 'learning_rate': 1.9242266917188943e-05, 'batcher': True}. Best is trial 1 with value: -5.1347054137315284.
[I 2023-12-27 16:23:58,178] Trial 3 finished with value: -0.39048636800612624 and parameters: {'hidden_dim': 272, 'k': 4, 'learning_rate': 0.0013174103144958104, 'batcher': False}. Best is trial 3 with value: -0.39048636800612624.
[I 2023-12-27 16:23:59,107] Trial 4 finished with value: -67.35011136698775 and paramet

In [15]:
study.trials_dataframe().to_csv(f"csvs/single-node_t-{node_type}_n-{size_elem}_normalized-{normalized}_post-normalized-{post_normalized}_dataset-size-{dataset_size}.csv")

In [16]:
size_elem = 10
n_trials = 200
dataset_size = 100
normalized = False
post_normalized = True
study = optuna.create_study(study_name=f"Study node type {node_type}, normalized: {normalized}, post-normalized:{post_normalized}, dataset size: {dataset_size}", directions=['maximize'])
study.optimize(objective, n_trials=n_trials, show_progress_bar=True)

[I 2023-12-27 16:27:14,868] A new study created in memory with name: Study node type S, normalized: False, post-normalized:True, dataset size: 100


  0%|          | 0/200 [00:00<?, ?it/s]

[I 2023-12-27 16:27:15,861] Trial 0 finished with value: -1.1608665412183332 and parameters: {'hidden_dim': 176, 'k': 1, 'learning_rate': 0.0024091880169727765, 'batcher': True}. Best is trial 0 with value: -1.1608665412183332.
[I 2023-12-27 16:27:17,198] Trial 1 finished with value: -3.6624973561169867 and parameters: {'hidden_dim': 288, 'k': 4, 'learning_rate': 0.007987667320208012, 'batcher': True}. Best is trial 0 with value: -1.1608665412183332.
[I 2023-12-27 16:27:18,205] Trial 2 finished with value: -1.7062378541737204 and parameters: {'hidden_dim': 304, 'k': 1, 'learning_rate': 0.0009539502920972615, 'batcher': False}. Best is trial 0 with value: -1.1608665412183332.
[I 2023-12-27 16:27:19,313] Trial 3 finished with value: -1.9384572226697414 and parameters: {'hidden_dim': 304, 'k': 2, 'learning_rate': 0.00013900280470150646, 'batcher': False}. Best is trial 0 with value: -1.1608665412183332.
[I 2023-12-27 16:27:20,751] Trial 4 finished with value: -126.24047045573093 and param

In [17]:
study.trials_dataframe().to_csv(f"csvs/single-node_t-{node_type}_n-{size_elem}_normalized-{normalized}_post-normalized-{post_normalized}_dataset-size-{dataset_size}.csv")

#### Data set size 1000

In [18]:
size_elem = 10
n_trials = 50
dataset_size = 1000
normalized = False
post_normalized = False
study = optuna.create_study(study_name=f"Study node type {node_type}, normalized: {normalized}, post-normalized:{post_normalized}, dataset size: {dataset_size}", directions=['maximize'])
study.optimize(objective, n_trials=n_trials, show_progress_bar=True)

[I 2023-12-27 16:31:11,862] A new study created in memory with name: Study node type S, normalized: False, post-normalized:False, dataset size: 1000


  0%|          | 0/50 [00:00<?, ?it/s]


100%|██████████| 1000/1000 [00:00<00:00, 111122.11it/s]

100%|██████████| 1000/1000 [00:03<00:00, 253.32it/s][A


[I 2023-12-27 16:31:24,328] Trial 0 finished with value: -184.546177783155 and parameters: {'hidden_dim': 160, 'k': 1, 'learning_rate': 0.006836302091855041, 'batcher': False}. Best is trial 0 with value: -184.546177783155.
[I 2023-12-27 16:31:33,399] Trial 1 finished with value: -33.82535948631753 and parameters: {'hidden_dim': 160, 'k': 3, 'learning_rate': 0.00031901411374522044, 'batcher': True}. Best is trial 1 with value: -33.82535948631753.
[I 2023-12-27 16:31:41,023] Trial 2 finished with value: -251.82813287987113 and parameters: {'hidden_dim': 48, 'k': 3, 'learning_rate': 0.0028961265306152195, 'batcher': True}. Best is trial 1 with value: -33.82535948631753.
[I 2023-12-27 16:31:49,078] Trial 3 finished with value: -4070.164029286003 and parameters: {'hidden_dim': 176, 'k': 2, 'learning_rate': 0.004417283854052826, 'batcher': True}. Best is trial 1 with value: -33.82535948631753.
[I 2023-12-27 16:31:57,366] Trial 4 finished with value: -2754.5644191375404 and parameters: {'hid

In [19]:
study.trials_dataframe().to_csv(f"csvs/single-node_t-{node_type}_n-{size_elem}_normalized-{normalized}_post-normalized-{post_normalized}_dataset-size-{dataset_size}.csv")

In [20]:
size_elem = 10
n_trials = 100
dataset_size = 1000
normalized = True
post_normalized = False
study = optuna.create_study(study_name=f"Study node type {node_type}, normalized: {normalized}, post-normalized:{post_normalized}, dataset size: {dataset_size}", directions=['maximize'])
study.optimize(objective, n_trials=n_trials, show_progress_bar=True)

[I 2023-12-27 16:38:19,425] A new study created in memory with name: Study node type S, normalized: True, post-normalized:False, dataset size: 1000


  0%|          | 0/100 [00:00<?, ?it/s]


100%|██████████| 1000/1000 [00:00<00:00, 111113.28it/s]

100%|██████████| 1000/1000 [00:05<00:00, 198.89it/s][A


[I 2023-12-27 16:38:31,997] Trial 0 finished with value: -161.8857669949863 and parameters: {'hidden_dim': 352, 'k': 3, 'learning_rate': 1.8306770726359294e-05, 'batcher': True}. Best is trial 0 with value: -161.8857669949863.
[I 2023-12-27 16:38:38,423] Trial 1 finished with value: -187.05431307465892 and parameters: {'hidden_dim': 96, 'k': 1, 'learning_rate': 1.2253757633483976e-05, 'batcher': True}. Best is trial 0 with value: -161.8857669949863.
[I 2023-12-27 16:38:44,751] Trial 2 finished with value: -183.95944934404977 and parameters: {'hidden_dim': 48, 'k': 2, 'learning_rate': 0.00012867334263816236, 'batcher': False}. Best is trial 0 with value: -161.8857669949863.
[I 2023-12-27 16:38:51,608] Trial 3 finished with value: -139.359082026909 and parameters: {'hidden_dim': 128, 'k': 4, 'learning_rate': 0.00011476242603557578, 'batcher': True}. Best is trial 3 with value: -139.359082026909.
[I 2023-12-27 16:38:58,131] Trial 4 finished with value: -7.51071846463163 and parameters: {'

In [21]:
study.trials_dataframe().to_csv(f"csvs/single-node_t-{node_type}_n-{size_elem}_normalized-{normalized}_post-normalized-{post_normalized}_dataset-size-{dataset_size}.csv")

In [22]:
size_elem = 10
n_trials = 150
dataset_size = 1000
normalized = False
post_normalized = True
study = optuna.create_study(study_name=f"Study node type {node_type}, normalized: {normalized}, post-normalized:{post_normalized}, dataset size: {dataset_size}", directions=['maximize'])
study.optimize(objective, n_trials=n_trials, show_progress_bar=True)

[I 2023-12-27 16:50:01,605] A new study created in memory with name: Study node type S, normalized: False, post-normalized:True, dataset size: 1000


  0%|          | 0/150 [00:00<?, ?it/s]

[I 2023-12-27 16:50:09,992] Trial 0 finished with value: -2.5230528775556573 and parameters: {'hidden_dim': 32, 'k': 4, 'learning_rate': 0.0023450500484111622, 'batcher': True}. Best is trial 0 with value: -2.5230528775556573.
[I 2023-12-27 16:50:19,054] Trial 1 finished with value: -4.5915865882688065 and parameters: {'hidden_dim': 512, 'k': 1, 'learning_rate': 0.0062980408433451545, 'batcher': False}. Best is trial 0 with value: -2.5230528775556573.
[I 2023-12-27 16:50:27,629] Trial 2 finished with value: -161.47470599840995 and parameters: {'hidden_dim': 336, 'k': 2, 'learning_rate': 2.7120565898650146e-05, 'batcher': True}. Best is trial 0 with value: -2.5230528775556573.
[I 2023-12-27 16:50:35,865] Trial 3 finished with value: -7.037187481365551 and parameters: {'hidden_dim': 64, 'k': 2, 'learning_rate': 0.0020214055814518285, 'batcher': True}. Best is trial 0 with value: -2.5230528775556573.
[I 2023-12-27 16:50:44,846] Trial 4 finished with value: -151.614864628235 and parameters

In [23]:
study.trials_dataframe().to_csv(f"csvs/single-node_t-{node_type}_n-{size_elem}_normalized-{normalized}_post-normalized-{post_normalized}_dataset-size-{dataset_size}.csv")

### Q

In [24]:
node_type = "Q"

#### Data set size 10

In [25]:
size_elem = 10
n_trials = 50
dataset_size = 10
normalized = False
post_normalized = False
study = optuna.create_study(study_name=f"Study node type {node_type}, normalized: {normalized}, post-normalized:{post_normalized}, dataset size: {dataset_size}", directions=['maximize'])
study.optimize(objective, n_trials=n_trials, show_progress_bar=True)

[I 2023-12-27 17:14:03,194] A new study created in memory with name: Study node type Q, normalized: False, post-normalized:False, dataset size: 10


  0%|          | 0/50 [00:00<?, ?it/s]


100%|██████████| 10/10 [00:00<00:00, 9998.34it/s]

100%|██████████| 10/10 [00:00<00:00, 158.73it/s]


[I 2023-12-27 17:14:06,060] Trial 0 finished with value: -12966.588892185566 and parameters: {'hidden_dim': 496, 'k': 2, 'learning_rate': 0.0017247127415270847, 'batcher': False}. Best is trial 0 with value: -12966.588892185566.
[I 2023-12-27 17:14:06,282] Trial 1 finished with value: -709.9210134784871 and parameters: {'hidden_dim': 400, 'k': 2, 'learning_rate': 2.7577768863407027e-05, 'batcher': True}. Best is trial 1 with value: -709.9210134784871.
[I 2023-12-27 17:14:06,508] Trial 2 finished with value: -616.0971490879848 and parameters: {'hidden_dim': 432, 'k': 2, 'learning_rate': 0.0003529918392352455, 'batcher': False}. Best is trial 2 with value: -616.0971490879848.
[I 2023-12-27 17:14:06,710] Trial 3 finished with value: -379.07981049195985 and parameters: {'hidden_dim': 192, 'k': 1, 'learning_rate': 0.0044864910649456535, 'batcher': False}. Best is trial 3 with value: -379.07981049195985.
[I 2023-12-27 17:14:06,916] Trial 4 finished with value: -129.36736576151088 and paramet

In [26]:
study.trials_dataframe().to_csv(f"csvs/single-node_t-{node_type}_n-{size_elem}_normalized-{normalized}_post-normalized-{post_normalized}_dataset-size-{dataset_size}.csv")

In [27]:
size_elem = 10
n_trials = 200
dataset_size = 10
normalized = True
post_normalized = False
study = optuna.create_study(study_name=f"Study node type {node_type}, normalized: {normalized}, post-normalized:{post_normalized}, dataset size: {dataset_size}", directions=['maximize'])
study.optimize(objective, n_trials=n_trials, show_progress_bar=True)

[I 2023-12-27 17:14:19,830] A new study created in memory with name: Study node type Q, normalized: True, post-normalized:False, dataset size: 10


  0%|          | 0/200 [00:00<?, ?it/s]


100%|██████████| 10/10 [00:00<?, ?it/s][A

100%|██████████| 10/10 [00:00<00:00, 175.44it/s]


[I 2023-12-27 17:14:22,772] Trial 0 finished with value: -5177.730861172124 and parameters: {'hidden_dim': 496, 'k': 5, 'learning_rate': 3.155934699984368e-05, 'batcher': False}. Best is trial 0 with value: -5177.730861172124.
[I 2023-12-27 17:14:22,984] Trial 1 finished with value: -153.03323616225032 and parameters: {'hidden_dim': 400, 'k': 1, 'learning_rate': 0.007436967481864274, 'batcher': False}. Best is trial 1 with value: -153.03323616225032.
[I 2023-12-27 17:14:23,267] Trial 2 finished with value: -59.15694370395643 and parameters: {'hidden_dim': 480, 'k': 5, 'learning_rate': 0.0006021762029336841, 'batcher': False}. Best is trial 2 with value: -59.15694370395643.
[I 2023-12-27 17:14:23,479] Trial 3 finished with value: -49.31337411478965 and parameters: {'hidden_dim': 144, 'k': 1, 'learning_rate': 0.008298612165369871, 'batcher': False}. Best is trial 3 with value: -49.31337411478965.
[I 2023-12-27 17:14:23,750] Trial 4 finished with value: -447.9840436520334 and parameters: 

In [28]:
study.trials_dataframe().to_csv(f"csvs/single-node_t-{node_type}_n-{size_elem}_normalized-{normalized}_post-normalized-{post_normalized}_dataset-size-{dataset_size}.csv")

In [29]:
size_elem = 10
n_trials = 200
dataset_size = 10
normalized = False
post_normalized = True
study = optuna.create_study(study_name=f"Study node type {node_type}, normalized: {normalized}, post-normalized:{post_normalized}, dataset size: {dataset_size}", directions=['maximize'])
study.optimize(objective, n_trials=n_trials, show_progress_bar=True)

[I 2023-12-27 17:15:17,053] A new study created in memory with name: Study node type Q, normalized: False, post-normalized:True, dataset size: 10


  0%|          | 0/200 [00:00<?, ?it/s]

[I 2023-12-27 17:15:17,341] Trial 0 finished with value: -217.36588814786177 and parameters: {'hidden_dim': 208, 'k': 4, 'learning_rate': 3.381289889578683e-05, 'batcher': True}. Best is trial 0 with value: -217.36588814786177.
[I 2023-12-27 17:15:17,764] Trial 1 finished with value: -7.830172050471871 and parameters: {'hidden_dim': 176, 'k': 5, 'learning_rate': 0.0005532433522758135, 'batcher': True}. Best is trial 1 with value: -7.830172050471871.
[I 2023-12-27 17:15:18,088] Trial 2 finished with value: -227.81039803997137 and parameters: {'hidden_dim': 400, 'k': 1, 'learning_rate': 2.27937183423115e-05, 'batcher': False}. Best is trial 1 with value: -7.830172050471871.
[I 2023-12-27 17:15:18,330] Trial 3 finished with value: -0.37501114851149414 and parameters: {'hidden_dim': 224, 'k': 2, 'learning_rate': 0.001799889834632253, 'batcher': False}. Best is trial 3 with value: -0.37501114851149414.
[I 2023-12-27 17:15:18,632] Trial 4 finished with value: -12.582389482915191 and paramete

In [30]:
study.trials_dataframe().to_csv(f"csvs/single-node_t-{node_type}_n-{size_elem}_normalized-{normalized}_post-normalized-{post_normalized}_dataset-size-{dataset_size}.csv")

#### Data set size 100

In [31]:
size_elem = 10
n_trials = 50
dataset_size = 100
normalized = False
post_normalized = False
study = optuna.create_study(study_name=f"Study node type {node_type}, normalized: {normalized}, post-normalized:{post_normalized}, dataset size: {dataset_size}", directions=['maximize'])
study.optimize(objective, n_trials=n_trials, show_progress_bar=True)

[I 2023-12-27 17:16:12,000] A new study created in memory with name: Study node type Q, normalized: False, post-normalized:False, dataset size: 100


  0%|          | 0/50 [00:00<?, ?it/s]


100%|██████████| 100/100 [00:00<00:00, 81617.12it/s]

100%|██████████| 100/100 [00:03<00:00, 30.12it/s]


[I 2023-12-27 17:16:17,110] Trial 0 finished with value: -44436.19118245831 and parameters: {'hidden_dim': 192, 'k': 4, 'learning_rate': 1.1245866242508752e-05, 'batcher': False}. Best is trial 0 with value: -44436.19118245831.
[I 2023-12-27 17:16:17,977] Trial 1 finished with value: -1187.127434858759 and parameters: {'hidden_dim': 272, 'k': 2, 'learning_rate': 0.00042860354962205544, 'batcher': False}. Best is trial 1 with value: -1187.127434858759.
[I 2023-12-27 17:16:18,885] Trial 2 finished with value: -787.614226141214 and parameters: {'hidden_dim': 128, 'k': 5, 'learning_rate': 1.1925065551912673e-05, 'batcher': False}. Best is trial 2 with value: -787.614226141214.
[I 2023-12-27 17:16:19,905] Trial 3 finished with value: -96.3544321479354 and parameters: {'hidden_dim': 368, 'k': 4, 'learning_rate': 3.124045076724744e-05, 'batcher': False}. Best is trial 3 with value: -96.3544321479354.
[I 2023-12-27 17:16:20,938] Trial 4 finished with value: -438.18660389174113 and parameters: 

In [32]:
study.trials_dataframe().to_csv(f"csvs/single-node_t-{node_type}_n-{size_elem}_normalized-{normalized}_post-normalized-{post_normalized}_dataset-size-{dataset_size}.csv")

In [33]:
size_elem = 10
n_trials = 200
dataset_size = 100
normalized = True
post_normalized = False
study = optuna.create_study(study_name=f"Study node type {node_type}, normalized: {normalized}, post-normalized:{post_normalized}, dataset size: {dataset_size}", directions=['maximize'])
study.optimize(objective, n_trials=n_trials, show_progress_bar=True)

[I 2023-12-27 17:17:06,035] A new study created in memory with name: Study node type Q, normalized: True, post-normalized:False, dataset size: 100


  0%|          | 0/200 [00:00<?, ?it/s]


100%|██████████| 100/100 [00:00<00:00, 49985.75it/s]

100%|██████████| 100/100 [00:03<00:00, 30.71it/s][A


[I 2023-12-27 17:17:10,893] Trial 0 finished with value: -142.62819376157805 and parameters: {'hidden_dim': 160, 'k': 5, 'learning_rate': 5.61333322268373e-05, 'batcher': True}. Best is trial 0 with value: -142.62819376157805.
[I 2023-12-27 17:17:11,665] Trial 1 finished with value: -1.446844992674749 and parameters: {'hidden_dim': 96, 'k': 4, 'learning_rate': 0.002612703832221748, 'batcher': False}. Best is trial 1 with value: -1.446844992674749.
[I 2023-12-27 17:17:12,477] Trial 2 finished with value: 0.5168939823959797 and parameters: {'hidden_dim': 512, 'k': 3, 'learning_rate': 0.0015129419922820667, 'batcher': True}. Best is trial 2 with value: 0.5168939823959797.
[I 2023-12-27 17:17:13,256] Trial 3 finished with value: -7.537112106962058 and parameters: {'hidden_dim': 48, 'k': 4, 'learning_rate': 0.0026549923830437895, 'batcher': False}. Best is trial 2 with value: 0.5168939823959797.
[I 2023-12-27 17:17:14,053] Trial 4 finished with value: -150.24631364171506 and parameters: {'h

In [34]:
study.trials_dataframe().to_csv(f"csvs/single-node_t-{node_type}_n-{size_elem}_normalized-{normalized}_post-normalized-{post_normalized}_dataset-size-{dataset_size}.csv")

In [35]:
size_elem = 10
n_trials = 200
dataset_size = 100
normalized = False
post_normalized = True
study = optuna.create_study(study_name=f"Study node type {node_type}, normalized: {normalized}, post-normalized:{post_normalized}, dataset size: {dataset_size}", directions=['maximize'])
study.optimize(objective, n_trials=n_trials, show_progress_bar=True)

[I 2023-12-27 17:20:22,733] A new study created in memory with name: Study node type Q, normalized: False, post-normalized:True, dataset size: 100


  0%|          | 0/200 [00:00<?, ?it/s]

[I 2023-12-27 17:20:23,863] Trial 0 finished with value: -140.77339509341965 and parameters: {'hidden_dim': 192, 'k': 5, 'learning_rate': 4.090178143070701e-05, 'batcher': True}. Best is trial 0 with value: -140.77339509341965.
[I 2023-12-27 17:20:24,888] Trial 1 finished with value: -156.56731543084396 and parameters: {'hidden_dim': 64, 'k': 4, 'learning_rate': 4.691836725520831e-05, 'batcher': False}. Best is trial 0 with value: -140.77339509341965.
[I 2023-12-27 17:20:26,161] Trial 2 finished with value: -161.36598813974314 and parameters: {'hidden_dim': 144, 'k': 3, 'learning_rate': 1.215779291414073e-05, 'batcher': True}. Best is trial 0 with value: -140.77339509341965.
[I 2023-12-27 17:20:27,114] Trial 3 finished with value: -4.7170137161757975 and parameters: {'hidden_dim': 176, 'k': 2, 'learning_rate': 0.00020009083695214343, 'batcher': False}. Best is trial 3 with value: -4.7170137161757975.
[I 2023-12-27 17:20:28,113] Trial 4 finished with value: -87.83679812817242 and parame

In [36]:
study.trials_dataframe().to_csv(f"csvs/single-node_t-{node_type}_n-{size_elem}_normalized-{normalized}_post-normalized-{post_normalized}_dataset-size-{dataset_size}.csv")

#### Data set size 1000

In [37]:
size_elem = 10
n_trials = 50
dataset_size = 1000
normalized = False
post_normalized = False
study = optuna.create_study(study_name=f"Study node type {node_type}, normalized: {normalized}, post-normalized:{post_normalized}, dataset size: {dataset_size}", directions=['maximize'])
study.optimize(objective, n_trials=n_trials, show_progress_bar=True)

[I 2023-12-27 17:24:25,752] A new study created in memory with name: Study node type Q, normalized: False, post-normalized:False, dataset size: 1000


  0%|          | 0/50 [00:00<?, ?it/s]


100%|██████████| 1000/1000 [00:00<00:00, 111128.00it/s]

100%|██████████| 1000/1000 [00:04<00:00, 237.47it/s][A


[I 2023-12-27 17:24:37,981] Trial 0 finished with value: -238.53173231796114 and parameters: {'hidden_dim': 80, 'k': 1, 'learning_rate': 0.003733883949588243, 'batcher': False}. Best is trial 0 with value: -238.53173231796114.
[I 2023-12-27 17:24:46,611] Trial 1 finished with value: -5.299646829592201 and parameters: {'hidden_dim': 384, 'k': 5, 'learning_rate': 0.00618311221779344, 'batcher': True}. Best is trial 1 with value: -5.299646829592201.
[I 2023-12-27 17:24:55,185] Trial 2 finished with value: -273.548795411413 and parameters: {'hidden_dim': 288, 'k': 4, 'learning_rate': 0.00019498470985326361, 'batcher': True}. Best is trial 1 with value: -5.299646829592201.
[I 2023-12-27 17:25:04,295] Trial 3 finished with value: -477.6969918091737 and parameters: {'hidden_dim': 416, 'k': 4, 'learning_rate': 0.0003051135490767406, 'batcher': True}. Best is trial 1 with value: -5.299646829592201.
[I 2023-12-27 17:25:12,661] Trial 4 finished with value: -95.15104837664 and parameters: {'hidden

In [38]:
study.trials_dataframe().to_csv(f"csvs/single-node_t-{node_type}_n-{size_elem}_normalized-{normalized}_post-normalized-{post_normalized}_dataset-size-{dataset_size}.csv")

In [39]:
size_elem = 10
n_trials = 100
dataset_size = 1000
normalized = True
post_normalized = False
study = optuna.create_study(study_name=f"Study node type {node_type}, normalized: {normalized}, post-normalized:{post_normalized}, dataset size: {dataset_size}", directions=['maximize'])
study.optimize(objective, n_trials=n_trials, show_progress_bar=True)

[I 2023-12-27 17:31:25,750] A new study created in memory with name: Study node type Q, normalized: True, post-normalized:False, dataset size: 1000


  0%|          | 0/100 [00:00<?, ?it/s]


100%|██████████| 1000/1000 [00:00<00:00, 166725.13it/s]

100%|██████████| 1000/1000 [00:04<00:00, 220.79it/s][A


[I 2023-12-27 17:31:37,357] Trial 0 finished with value: -0.4543552347556538 and parameters: {'hidden_dim': 112, 'k': 3, 'learning_rate': 0.005092846608145021, 'batcher': True}. Best is trial 0 with value: -0.4543552347556538.
[I 2023-12-27 17:31:44,803] Trial 1 finished with value: -8.05565289138039 and parameters: {'hidden_dim': 400, 'k': 2, 'learning_rate': 8.580019632707998e-05, 'batcher': False}. Best is trial 0 with value: -0.4543552347556538.
[I 2023-12-27 17:31:51,660] Trial 2 finished with value: -167.1278029422257 and parameters: {'hidden_dim': 64, 'k': 4, 'learning_rate': 3.7489074061635906e-05, 'batcher': False}. Best is trial 0 with value: -0.4543552347556538.
[I 2023-12-27 17:31:58,366] Trial 3 finished with value: -152.5794321140266 and parameters: {'hidden_dim': 144, 'k': 2, 'learning_rate': 6.826637163623124e-05, 'batcher': True}. Best is trial 0 with value: -0.4543552347556538.
[I 2023-12-27 17:32:05,760] Trial 4 finished with value: -5.768433994101864 and parameters:

In [40]:
study.trials_dataframe().to_csv(f"csvs/single-node_t-{node_type}_n-{size_elem}_normalized-{normalized}_post-normalized-{post_normalized}_dataset-size-{dataset_size}.csv")

In [41]:
size_elem = 10
n_trials = 150
dataset_size = 1000
normalized = False
post_normalized = True
study = optuna.create_study(study_name=f"Study node type {node_type}, normalized: {normalized}, post-normalized:{post_normalized}, dataset size: {dataset_size}", directions=['maximize'])
study.optimize(objective, n_trials=n_trials, show_progress_bar=True)

[I 2023-12-27 17:43:44,689] A new study created in memory with name: Study node type Q, normalized: False, post-normalized:True, dataset size: 1000


  0%|          | 0/150 [00:00<?, ?it/s]

[I 2023-12-27 17:43:53,959] Trial 0 finished with value: -154.98786128309888 and parameters: {'hidden_dim': 352, 'k': 1, 'learning_rate': 1.9753158948336232e-05, 'batcher': False}. Best is trial 0 with value: -154.98786128309888.
[I 2023-12-27 17:44:03,988] Trial 1 finished with value: -1.3103712488881059 and parameters: {'hidden_dim': 256, 'k': 5, 'learning_rate': 0.00046677126037819133, 'batcher': True}. Best is trial 1 with value: -1.3103712488881059.
[I 2023-12-27 17:44:14,131] Trial 2 finished with value: -1.9088195513453752 and parameters: {'hidden_dim': 496, 'k': 4, 'learning_rate': 0.0002924620074661427, 'batcher': False}. Best is trial 1 with value: -1.3103712488881059.
[I 2023-12-27 17:44:24,089] Trial 3 finished with value: -104.70543690750617 and parameters: {'hidden_dim': 368, 'k': 2, 'learning_rate': 4.844548063331315e-05, 'batcher': False}. Best is trial 1 with value: -1.3103712488881059.
[I 2023-12-27 17:44:33,650] Trial 4 finished with value: -119.77029504950863 and pa

In [42]:
study.trials_dataframe().to_csv(f"csvs/single-node_t-{node_type}_n-{size_elem}_normalized-{normalized}_post-normalized-{post_normalized}_dataset-size-{dataset_size}.csv")

### C

In [43]:
node_type = "C"

#### Data set size 10

In [44]:
size_elem = 10
n_trials = 50
dataset_size = 10
normalized = False
post_normalized = False
study = optuna.create_study(study_name=f"Study node type {node_type}, normalized: {normalized}, post-normalized:{post_normalized}, dataset size: {dataset_size}", directions=['maximize'])
study.optimize(objective, n_trials=n_trials, show_progress_bar=True)

[I 2023-12-27 18:08:11,417] A new study created in memory with name: Study node type C, normalized: False, post-normalized:False, dataset size: 10


  0%|          | 0/50 [00:00<?, ?it/s]


100%|██████████| 10/10 [00:00<?, ?it/s][A

100%|██████████| 10/10 [00:00<00:00, 153.83it/s]


[I 2023-12-27 18:08:14,450] Trial 0 finished with value: -2278.1854599402473 and parameters: {'hidden_dim': 288, 'k': 5, 'learning_rate': 0.0010499208304213493, 'batcher': True}. Best is trial 0 with value: -2278.1854599402473.
[I 2023-12-27 18:08:14,740] Trial 1 finished with value: -278.1452855034797 and parameters: {'hidden_dim': 240, 'k': 2, 'learning_rate': 0.0007784960105237188, 'batcher': False}. Best is trial 1 with value: -278.1452855034797.
[I 2023-12-27 18:08:14,976] Trial 2 finished with value: -165.11894640432914 and parameters: {'hidden_dim': 128, 'k': 1, 'learning_rate': 1.5075031070406312e-05, 'batcher': True}. Best is trial 2 with value: -165.11894640432914.
[I 2023-12-27 18:08:15,250] Trial 3 finished with value: -270.14281386496526 and parameters: {'hidden_dim': 96, 'k': 1, 'learning_rate': 0.006195495595430689, 'batcher': True}. Best is trial 2 with value: -165.11894640432914.
[I 2023-12-27 18:08:15,530] Trial 4 finished with value: -18.24254364950624 and parameters

In [45]:
study.trials_dataframe().to_csv(f"csvs/single-node_t-{node_type}_n-{size_elem}_normalized-{normalized}_post-normalized-{post_normalized}_dataset-size-{dataset_size}.csv")

In [46]:
size_elem = 10
n_trials = 200
dataset_size = 10
normalized = True
post_normalized = False
study = optuna.create_study(study_name=f"Study node type {node_type}, normalized: {normalized}, post-normalized:{post_normalized}, dataset size: {dataset_size}", directions=['maximize'])
study.optimize(objective, n_trials=n_trials, show_progress_bar=True)

[I 2023-12-27 18:08:29,278] A new study created in memory with name: Study node type C, normalized: True, post-normalized:False, dataset size: 10


  0%|          | 0/200 [00:00<?, ?it/s]


100%|██████████| 10/10 [00:00<00:00, 10015.05it/s]

100%|██████████| 10/10 [00:00<00:00, 224.58it/s]


[I 2023-12-27 18:08:32,096] Trial 0 finished with value: -641.3493138763528 and parameters: {'hidden_dim': 256, 'k': 4, 'learning_rate': 1.408875004428096e-05, 'batcher': False}. Best is trial 0 with value: -641.3493138763528.
[I 2023-12-27 18:08:32,305] Trial 1 finished with value: -682.3757729133551 and parameters: {'hidden_dim': 208, 'k': 1, 'learning_rate': 4.734906596428063e-05, 'batcher': False}. Best is trial 0 with value: -641.3493138763528.
[I 2023-12-27 18:08:32,623] Trial 2 finished with value: -15.900593574174218 and parameters: {'hidden_dim': 480, 'k': 3, 'learning_rate': 0.00047531645909922276, 'batcher': False}. Best is trial 2 with value: -15.900593574174218.
[I 2023-12-27 18:08:32,880] Trial 3 finished with value: -13.018382409272476 and parameters: {'hidden_dim': 496, 'k': 3, 'learning_rate': 0.001982213795762967, 'batcher': True}. Best is trial 3 with value: -13.018382409272476.
[I 2023-12-27 18:08:33,247] Trial 4 finished with value: -728.4533926602768 and parameter

In [47]:
study.trials_dataframe().to_csv(f"csvs/single-node_t-{node_type}_n-{size_elem}_normalized-{normalized}_post-normalized-{post_normalized}_dataset-size-{dataset_size}.csv")

In [48]:
size_elem = 10
n_trials = 200
dataset_size = 10
normalized = False
post_normalized = True
study = optuna.create_study(study_name=f"Study node type {node_type}, normalized: {normalized}, post-normalized:{post_normalized}, dataset size: {dataset_size}", directions=['maximize'])
study.optimize(objective, n_trials=n_trials, show_progress_bar=True)

[I 2023-12-27 18:09:33,492] A new study created in memory with name: Study node type C, normalized: False, post-normalized:True, dataset size: 10


  0%|          | 0/200 [00:00<?, ?it/s]

[I 2023-12-27 18:09:33,839] Trial 0 finished with value: -2.236714551798744 and parameters: {'hidden_dim': 64, 'k': 4, 'learning_rate': 0.0030874806434115177, 'batcher': True}. Best is trial 0 with value: -2.236714551798744.
[I 2023-12-27 18:09:34,074] Trial 1 finished with value: -109.03812266877158 and parameters: {'hidden_dim': 288, 'k': 1, 'learning_rate': 9.570306116386424e-05, 'batcher': False}. Best is trial 0 with value: -2.236714551798744.
[I 2023-12-27 18:09:34,387] Trial 2 finished with value: -0.7964168803429428 and parameters: {'hidden_dim': 80, 'k': 1, 'learning_rate': 0.0027259100816369166, 'batcher': False}. Best is trial 2 with value: -0.7964168803429428.
[I 2023-12-27 18:09:34,616] Trial 3 finished with value: -133.0925985863466 and parameters: {'hidden_dim': 256, 'k': 1, 'learning_rate': 1.8548566277552477e-05, 'batcher': True}. Best is trial 2 with value: -0.7964168803429428.
[I 2023-12-27 18:09:34,878] Trial 4 finished with value: -137.49220227580645 and parameters

In [49]:
study.trials_dataframe().to_csv(f"csvs/single-node_t-{node_type}_n-{size_elem}_normalized-{normalized}_post-normalized-{post_normalized}_dataset-size-{dataset_size}.csv")

#### Data set size 100

In [50]:
size_elem = 10
n_trials = 50
dataset_size = 100
normalized = False
post_normalized = False
study = optuna.create_study(study_name=f"Study node type {node_type}, normalized: {normalized}, post-normalized:{post_normalized}, dataset size: {dataset_size}", directions=['maximize'])
study.optimize(objective, n_trials=n_trials, show_progress_bar=True)

[I 2023-12-27 18:10:43,259] A new study created in memory with name: Study node type C, normalized: False, post-normalized:False, dataset size: 100


  0%|          | 0/50 [00:00<?, ?it/s]


100%|██████████| 100/100 [00:00<00:00, 100007.25it/s]

100%|██████████| 100/100 [00:03<00:00, 32.53it/s]


[I 2023-12-27 18:10:47,891] Trial 0 finished with value: -12160.990931715121 and parameters: {'hidden_dim': 464, 'k': 2, 'learning_rate': 0.0015735845566663892, 'batcher': False}. Best is trial 0 with value: -12160.990931715121.
[I 2023-12-27 18:10:48,887] Trial 1 finished with value: -16016.626813133782 and parameters: {'hidden_dim': 240, 'k': 5, 'learning_rate': 0.0016147263274114746, 'batcher': True}. Best is trial 0 with value: -12160.990931715121.
[I 2023-12-27 18:10:49,913] Trial 2 finished with value: -87492.11608053588 and parameters: {'hidden_dim': 368, 'k': 4, 'learning_rate': 0.003870996218417288, 'batcher': True}. Best is trial 0 with value: -12160.990931715121.
[I 2023-12-27 18:10:50,779] Trial 3 finished with value: -23.891307915421915 and parameters: {'hidden_dim': 112, 'k': 1, 'learning_rate': 0.00017047131420460472, 'batcher': False}. Best is trial 3 with value: -23.891307915421915.
[I 2023-12-27 18:10:51,670] Trial 4 finished with value: -95.34199003005493 and paramet

In [51]:
study.trials_dataframe().to_csv(f"csvs/single-node_t-{node_type}_n-{size_elem}_normalized-{normalized}_post-normalized-{post_normalized}_dataset-size-{dataset_size}.csv")

In [52]:
size_elem = 10
n_trials = 200
dataset_size = 100
normalized = True
post_normalized = False
study = optuna.create_study(study_name=f"Study node type {node_type}, normalized: {normalized}, post-normalized:{post_normalized}, dataset size: {dataset_size}", directions=['maximize'])
study.optimize(objective, n_trials=n_trials, show_progress_bar=True)

[I 2023-12-27 18:11:32,879] A new study created in memory with name: Study node type C, normalized: True, post-normalized:False, dataset size: 100


  0%|          | 0/200 [00:00<?, ?it/s]


100%|██████████| 100/100 [00:00<00:00, 100007.25it/s]

100%|██████████| 100/100 [00:03<00:00, 30.00it/s][A


[I 2023-12-27 18:11:37,467] Trial 0 finished with value: -162.63282445622704 and parameters: {'hidden_dim': 80, 'k': 2, 'learning_rate': 1.2138639709542742e-05, 'batcher': False}. Best is trial 0 with value: -162.63282445622704.
[I 2023-12-27 18:11:38,366] Trial 1 finished with value: -2.1952582222715042 and parameters: {'hidden_dim': 368, 'k': 3, 'learning_rate': 0.003899216173505191, 'batcher': True}. Best is trial 1 with value: -2.1952582222715042.
[I 2023-12-27 18:11:39,173] Trial 2 finished with value: -30.399149126118186 and parameters: {'hidden_dim': 48, 'k': 2, 'learning_rate': 0.0008352137367125216, 'batcher': True}. Best is trial 1 with value: -2.1952582222715042.
[I 2023-12-27 18:11:40,096] Trial 3 finished with value: -0.13281800124401905 and parameters: {'hidden_dim': 336, 'k': 5, 'learning_rate': 0.0007471550604116849, 'batcher': False}. Best is trial 3 with value: -0.13281800124401905.
[I 2023-12-27 18:11:41,019] Trial 4 finished with value: -0.8360439812920868 and param

In [53]:
study.trials_dataframe().to_csv(f"csvs/single-node_t-{node_type}_n-{size_elem}_normalized-{normalized}_post-normalized-{post_normalized}_dataset-size-{dataset_size}.csv")

In [54]:
size_elem = 10
n_trials = 200
dataset_size = 100
normalized = False
post_normalized = True
study = optuna.create_study(study_name=f"Study node type {node_type}, normalized: {normalized}, post-normalized:{post_normalized}, dataset size: {dataset_size}", directions=['maximize'])
study.optimize(objective, n_trials=n_trials, show_progress_bar=True)

[I 2023-12-27 18:15:01,463] A new study created in memory with name: Study node type C, normalized: False, post-normalized:True, dataset size: 100


  0%|          | 0/200 [00:00<?, ?it/s]

[I 2023-12-27 18:15:02,847] Trial 0 finished with value: -6.027625281918934 and parameters: {'hidden_dim': 400, 'k': 3, 'learning_rate': 0.00023088677393566813, 'batcher': True}. Best is trial 0 with value: -6.027625281918934.
[I 2023-12-27 18:15:03,931] Trial 1 finished with value: -1.691177198693778 and parameters: {'hidden_dim': 80, 'k': 5, 'learning_rate': 0.0011447026533902868, 'batcher': False}. Best is trial 1 with value: -1.691177198693778.
[I 2023-12-27 18:15:04,963] Trial 2 finished with value: -185.14659082933628 and parameters: {'hidden_dim': 256, 'k': 2, 'learning_rate': 2.0308676186492354e-05, 'batcher': False}. Best is trial 1 with value: -1.691177198693778.
[I 2023-12-27 18:15:06,037] Trial 3 finished with value: -167.10638051620873 and parameters: {'hidden_dim': 288, 'k': 4, 'learning_rate': 1.792287183210738e-05, 'batcher': False}. Best is trial 1 with value: -1.691177198693778.
[I 2023-12-27 18:15:07,134] Trial 4 finished with value: -191.5049916872036 and parameters

In [55]:
study.trials_dataframe().to_csv(f"csvs/single-node_t-{node_type}_n-{size_elem}_normalized-{normalized}_post-normalized-{post_normalized}_dataset-size-{dataset_size}.csv")

#### Data set size 1000

In [56]:
size_elem = 10
n_trials = 50
dataset_size = 1000
normalized = False
post_normalized = False
study = optuna.create_study(study_name=f"Study node type {node_type}, normalized: {normalized}, post-normalized:{post_normalized}, dataset size: {dataset_size}", directions=['maximize'])
study.optimize(objective, n_trials=n_trials, show_progress_bar=True)

[I 2023-12-27 18:19:30,437] A new study created in memory with name: Study node type C, normalized: False, post-normalized:False, dataset size: 1000


  0%|          | 0/50 [00:00<?, ?it/s]


100%|██████████| 1000/1000 [00:00<00:00, 117520.43it/s]

100%|██████████| 1000/1000 [00:04<00:00, 214.27it/s][A


[I 2023-12-27 18:19:45,353] Trial 0 finished with value: -4988.974201243013 and parameters: {'hidden_dim': 240, 'k': 5, 'learning_rate': 0.0009288158356751166, 'batcher': False}. Best is trial 0 with value: -4988.974201243013.
[I 2023-12-27 18:19:55,391] Trial 1 finished with value: -829163.201616221 and parameters: {'hidden_dim': 384, 'k': 4, 'learning_rate': 0.005597723776733968, 'batcher': True}. Best is trial 0 with value: -4988.974201243013.
[I 2023-12-27 18:20:05,125] Trial 2 finished with value: -1325.071427253953 and parameters: {'hidden_dim': 384, 'k': 4, 'learning_rate': 5.8375863642549534e-05, 'batcher': True}. Best is trial 2 with value: -1325.071427253953.
[I 2023-12-27 18:20:15,157] Trial 3 finished with value: -598.2634355737133 and parameters: {'hidden_dim': 384, 'k': 5, 'learning_rate': 0.00016443858304075718, 'batcher': True}. Best is trial 3 with value: -598.2634355737133.
[I 2023-12-27 18:20:24,515] Trial 4 finished with value: -293.70272041516426 and parameters: {'

In [57]:
study.trials_dataframe().to_csv(f"csvs/single-node_t-{node_type}_n-{size_elem}_normalized-{normalized}_post-normalized-{post_normalized}_dataset-size-{dataset_size}.csv")

In [58]:
size_elem = 10
n_trials = 100
dataset_size = 1000
normalized = True
post_normalized = False
study = optuna.create_study(study_name=f"Study node type {node_type}, normalized: {normalized}, post-normalized:{post_normalized}, dataset size: {dataset_size}", directions=['maximize'])
study.optimize(objective, n_trials=n_trials, show_progress_bar=True)

[I 2023-12-27 18:27:39,242] A new study created in memory with name: Study node type C, normalized: True, post-normalized:False, dataset size: 1000


  0%|          | 0/100 [00:00<?, ?it/s]


100%|██████████| 1000/1000 [00:00<00:00, 92497.61it/s]

100%|██████████| 1000/1000 [00:05<00:00, 189.41it/s][A


[I 2023-12-27 18:27:53,768] Trial 0 finished with value: -193.68023463521277 and parameters: {'hidden_dim': 32, 'k': 3, 'learning_rate': 0.0003649679348218536, 'batcher': False}. Best is trial 0 with value: -193.68023463521277.
[I 2023-12-27 18:27:59,829] Trial 1 finished with value: -199.55788821127845 and parameters: {'hidden_dim': 112, 'k': 1, 'learning_rate': 4.423932469419764e-05, 'batcher': False}. Best is trial 0 with value: -193.68023463521277.
[I 2023-12-27 18:28:06,025] Trial 2 finished with value: -192.17702224239667 and parameters: {'hidden_dim': 224, 'k': 2, 'learning_rate': 1.5771598586382178e-05, 'batcher': True}. Best is trial 2 with value: -192.17702224239667.
[I 2023-12-27 18:28:12,464] Trial 3 finished with value: -2.180251975203882 and parameters: {'hidden_dim': 192, 'k': 5, 'learning_rate': 0.0035221979680830876, 'batcher': False}. Best is trial 3 with value: -2.180251975203882.
[I 2023-12-27 18:28:19,036] Trial 4 finished with value: -0.5440625777618873 and parame

In [59]:
study.trials_dataframe().to_csv(f"csvs/single-node_t-{node_type}_n-{size_elem}_normalized-{normalized}_post-normalized-{post_normalized}_dataset-size-{dataset_size}.csv")

In [60]:
size_elem = 10
n_trials = 150
dataset_size = 1000
normalized = False
post_normalized = True
study = optuna.create_study(study_name=f"Study node type {node_type}, normalized: {normalized}, post-normalized:{post_normalized}, dataset size: {dataset_size}", directions=['maximize'])
study.optimize(objective, n_trials=n_trials, show_progress_bar=True)

[I 2023-12-27 18:41:29,821] A new study created in memory with name: Study node type C, normalized: False, post-normalized:True, dataset size: 1000


  0%|          | 0/150 [00:00<?, ?it/s]

[I 2023-12-27 18:41:39,683] Trial 0 finished with value: -4.931431504566726 and parameters: {'hidden_dim': 464, 'k': 1, 'learning_rate': 0.00018523128936605431, 'batcher': True}. Best is trial 0 with value: -4.931431504566726.
[I 2023-12-27 18:41:49,947] Trial 1 finished with value: -2.3789128943509894 and parameters: {'hidden_dim': 352, 'k': 3, 'learning_rate': 0.002875429448843701, 'batcher': False}. Best is trial 1 with value: -2.3789128943509894.
[I 2023-12-27 18:41:58,522] Trial 2 finished with value: -228.27235688363257 and parameters: {'hidden_dim': 16, 'k': 4, 'learning_rate': 1.3440545870275656e-05, 'batcher': True}. Best is trial 1 with value: -2.3789128943509894.
[I 2023-12-27 18:42:07,750] Trial 3 finished with value: -166.73410707407024 and parameters: {'hidden_dim': 224, 'k': 2, 'learning_rate': 6.413245603587869e-05, 'batcher': True}. Best is trial 1 with value: -2.3789128943509894.
[I 2023-12-27 18:42:17,801] Trial 4 finished with value: -2.390318790457478 and parameter

In [61]:
study.trials_dataframe().to_csv(f"csvs/single-node_t-{node_type}_n-{size_elem}_normalized-{normalized}_post-normalized-{post_normalized}_dataset-size-{dataset_size}.csv")